This is a script that scrapes google scholar for all that juicy data.

First, it sets up a URL given a search topic you provide. It then searches for court cases with that topic
and filters only the links that are actual court cases. It uses the request module for this.

It then stores all those links in a list. It goes through those links and gets the HTML page of the court case.
Next, it makes use of BeautifulSoup, which has a built in way to remove all the HTML garbage, and gives us just text

In [1]:
import os
import requests
import re
import random
import time
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

Tricks the webpage into thinking i'm just an innocent chrome user

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

In [3]:
# Provide the name for the text file
def SimplifyCaseTitle(case_title):
    simplified_title = re.sub(r',\s\d+.*$', '', case_title)
    if len(simplified_title) > 128:
        return simplified_title[:128-3] + '..'
    else:
        return simplified_title

In [4]:
# Remove things like page breaks
def RemoveGarbage(DirtyString):
    cleaned_text = re.sub(r'\s+', ' ', DirtyString)
    return cleaned_text.strip()

In [5]:
# Send text to a text document in Output
def OutputText(text):
    FileName = SimplifyCaseTitle(text[0:300]) + ".txt"
    FileName = FileName.replace("\\", "")
    outputFolderPath = os.path.join(os.getcwd(), 'Output')
    if not os.path.exists(outputFolderPath):
        os.makedirs(outputFolderPath)
    filePath = os.path.join(outputFolderPath, FileName)
    with open(filePath, 'w', encoding='utf-8') as file:
        file.write(text)

In [6]:
# Takes a page num and search query and gives you a good URL
def GenerateBaseURL(SearchQuery, Page):
    SearchQuery = SearchQuery.replace(' ', '+')
    StartOn = (Page - 1) * 10
    return ScholarURL + PreStart + str(StartOn) + PreQuery + SearchQuery + PostQuery

In [7]:
# Find links on page and put them into a nice little list
def GetLinksFromPage(ParsedHTML):
    PageLinks = set()
    for link in ParsedHTML.find_all('a', href=True):
        full_url = urljoin(ScholarURL, link['href'])
        if urlparse(full_url).netloc == urlparse(ScholarURL).netloc:
            lowerURL = full_url.lower()
            IsGoodPage = 'scholar_case?case=' in lowerURL
            if IsGoodPage:
                PageLinks.add(full_url)
    return PageLinks

In [10]:
# Specific info for google scholar
ScholarURL = "https://scholar.google.com"
PreStart = "/scholar?start="
PreQuery = "&q="
PostQuery = "&hl=en&as_sdt=8000006"

# What you're searching for on the website
SearchQuery = "Microsoft"

# How many pages to parse? 1 page is 10 text documents.
# Too much of this gets me blocked. good thing i have a vpn on my laptop
HowManyPagesToParse = 6

In [11]:
TotalDocuments = 10 * HowManyPagesToParse
CurrentDocument = 1
for i in range(HowManyPagesToParse):
    # Go through the search results and find every webpage
    ParseTestURL = GenerateBaseURL(SearchQuery, i + 1)
    BaseSearchPage = requests.get(ParseTestURL, headers=headers)
    BaseSearchParse = BeautifulSoup(BaseSearchPage.text, 'html.parser')
    print("BASE URL: " + ParseTestURL)
    URLS = GetLinksFromPage(BaseSearchParse)

    for url in URLS:

        # So the overlords at google don't come kill me
        time.sleep(random.randint(1000, 4000) / 1000)
        
        # Go through every link on the search results, get text, and dump it into a file in the 'Output' folder
        Requested = requests.get(url, headers=headers)
        Parsed = BeautifulSoup(Requested.text, 'html.parser')
        for ScriptOrStyle in Parsed(["script", "style"]):
            ScriptOrStyle.decompose()
        TextFragments = [Element.get_text() for Element in Parsed.find_all(['p', 'h1', 'h2', 'h3', 'h4', 'h5', 'h6'])]
        CleanText = ' '.join(TextFragment.strip() for TextFragment in TextFragments)
        CleanText = RemoveGarbage(CleanText)
        print("(" + str(CurrentDocument) + "/" + str(TotalDocuments) + ") " + SimplifyCaseTitle(CleanText[0:300]) + ".txt")
        OutputText(CleanText)
        CurrentDocument += 1

BASE URL: https://scholar.google.com/scholar?start=0&q=Microsoft&hl=en&as_sdt=8000006
(1/60) US v. Microsoft Corp..txt
(2/60) Odom v. Microsoft Corp..txt
(3/60) US v. Microsoft Corp..txt
(4/60) US v. Microsoft Corp..txt
(5/60) US v. Microsoft Corp..txt
(6/60) Dickson v. Microsoft Corp..txt
(7/60) Vizcaino v. Microsoft Corp..txt
(8/60) Microsoft Corp. v. Baker.txt
(9/60) US v. Microsoft Corp..txt
(10/60) US v. Microsoft Corp..txt
BASE URL: https://scholar.google.com/scholar?start=10&q=Microsoft&hl=en&as_sdt=8000006
(11/60) Microsoft Corp. v. i4i Ltd. Partnership.txt
(12/60) Vizcaino v. Microsoft Corp..txt
(13/60) i4i Ltd. Partnership v. Microsoft Corp..txt
(14/60) ENFISH, LLC v. Microsoft Corp..txt
(15/60) Johnson v. Microsoft Corp..txt
(16/60) Massachusetts v. Microsoft Corp..txt
(17/60) Vizcaino v. Microsoft Corp..txt
(18/60) Amado v. Microsoft Corp..txt
(19/60) Microsoft Corp. v. Motorola, Inc..txt
(20/60) Microsoft Corp. v. AT & T CORP..txt
BASE URL: https://scholar.google.com/schol